In [1]:
import mindspore
from mindspore import nn, ops
from mindspore.dataset import MnistDataset, FashionMnistDataset, vision, transforms
from elastic_weight_consolidation import ElasticWeightConsolidation

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
image_transforms = [
                    vision.Rescale(1.0 / 255.0, 0),
                    #vision.Normalize(mean = (0.1307, ), std = (0.3081, )),
                    vision.HWC2CHW()
]
label_transform = transforms.TypeCast(mindspore.int32)
mnist_train = MnistDataset("MNIST_Data/train", usage = "train", shuffle = True)
mnist_train = mnist_train.map(image_transforms, 'image')
mnist_train = mnist_train.map(label_transform, 'label')
mnist_train = mnist_train.batch(100)
mnist_test = MnistDataset("MNIST_Data/test", usage = "test", shuffle = False)
mnist_test = mnist_test.map(image_transforms, 'image')
mnist_test = mnist_test.map(label_transform, 'label')
mnist_test = mnist_test.batch(100)

In [3]:
class LinearLayer(nn.Cell):
    
    def __init__(self, input_dim, output_dim, act = 'relu', use_bn = False):
        super(LinearLayer, self).__init__()
        self.use_bn = use_bn
        self.lin = nn.Dense(input_dim, output_dim)
        self.act = nn.ReLU() if act == 'relu' else act
        if use_bn:
            self.bn = nn.BatchNorm1d(output_dim)
            
    def construct(self, x):
        if self.use_bn:
            return self.bn(self.act(self.lin(x)))
        return self.act(self.lin(x))

class Flatten(nn.Cell):

    def construct(self, x):
        return x.view(x.shape[0], -1)

In [4]:
class BaseModel(nn.Cell):
    
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super(BaseModel, self).__init__()
        self.f1 = Flatten()
        self.lin1 = LinearLayer(num_inputs, num_hidden, use_bn = True)
        self.lin2 = LinearLayer(num_hidden, num_hidden, use_bn = True)
        self.lin3 = nn.Dense(num_hidden, num_outputs)
        
    def construct(self, x):
        return self.lin3(self.lin2(self.lin1(self.f1(x))))

In [5]:
crit = nn.CrossEntropyLoss()

In [6]:
ewc = ElasticWeightConsolidation(BaseModel(28 * 28, 100, 10), crit = crit, lr = 1e-4)

In [7]:
from tqdm import tqdm

In [8]:
for _ in range(4):
    for input, target in tqdm(mnist_train.create_tuple_iterator()):
        ewc.forward_backward_update(input, target)

600it [00:14, 40.11it/s]
600it [00:07, 77.80it/s]
600it [00:07, 76.44it/s]
600it [00:07, 78.07it/s]


In [9]:
ewc.register_ewc_params(mnist_train, 100, 300)

In [10]:
f_mnist_train = FashionMnistDataset("FashionMNIST_Data/train", usage = "train", shuffle = True)
f_mnist_train = f_mnist_train.map(image_transforms, 'image')
f_mnist_train = f_mnist_train.map(label_transform, 'label')
f_mnist_train = f_mnist_train.batch(100)
f_mnist_test = FashionMnistDataset("FashionMNIST_Data/test", usage = "test", shuffle = False)
f_mnist_test = f_mnist_test.map(image_transforms, 'image')
f_mnist_test = f_mnist_test.map(label_transform, 'label')
f_mnist_test = f_mnist_test.batch(100)

In [11]:
for _ in range(4):
    for input, target in tqdm(f_mnist_train.create_tuple_iterator()):
        ewc.forward_backward_update(input, target)

600it [00:17, 35.11it/s]
600it [00:16, 36.59it/s]
600it [00:16, 37.12it/s]
600it [00:15, 38.26it/s]


In [12]:
ewc.register_ewc_params(f_mnist_train, 100, 300)

In [13]:
def accu(model, dataset):
    model.set_train(False)
    acc = 0
    for input, target in dataset.create_tuple_iterator():
        o = model(input)
        acc += (o.argmax(axis = 1).long() == target).float().mean()
    return acc / dataset.get_batch_size()

In [14]:
accu(ewc.model, f_mnist_test)

Tensor(shape=[], dtype=Float32, value= 0.8171)

In [15]:
accu(ewc.model, mnist_test)

Tensor(shape=[], dtype=Float32, value= 0.649)